In [59]:
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack





In [60]:
!pip install -q sentence-transformers



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
data = pd.read_json(
    r"C:\Users\acer\Desktop\PROJECT\Dataset\problems_data.jsonl",
    lines=True
)

data["text"] = (
    data["title"].fillna("") + " " +
    data["description"].fillna("") + " " +
    data["input_description"].fillna("") + " " +
    data["output_description"].fillna("")
).str.lower()


In [62]:
data["io_text"] = (
    data["input_description"].fillna("") + " " +
    data["output_description"].fillna("")
).str.lower()


In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 
 1   description         4112 non-null   object 
 2   input_description   4112 non-null   object 
 3   output_description  4112 non-null   object 
 4   sample_io           4112 non-null   object 
 5   problem_class       4112 non-null   object 
 6   problem_score       4112 non-null   float64
 7   url                 4112 non-null   object 
 8   text                4112 non-null   object 
 9   io_text             4112 non-null   object 
dtypes: float64(1), object(9)
memory usage: 321.4+ KB


In [64]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data["problem_class_encoded"] = le.fit_transform(data["problem_class"])

y_class = data["problem_class_encoded"]


In [65]:
data.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url,text,io_text,problem_class_encoded
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu,uuu unununium (uuu) was the name of the chemic...,the input consists of one line with two intege...,1
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge,house building a number of eccentrics from cen...,"the input consists of $10$ test cases, which a...",1
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,,,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi,mario or luigi mario and luigi are playing a g...,,1
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3 3 C 2 C 1 C', 'output': 'GHOST...",hard,9.6,https://open.kattis.com/problems/thewireghost,the wire ghost žofka is bending a copper wire....,the first line contains two integers $l$ and $...,1
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0 10 0 10 10', 'output': '14.14'...",hard,9.6,https://open.kattis.com/problems/barktree,barking up the wrong tree your dog spot is let...,the first line of input consists of two intege...,1


In [66]:
y_class.head()

0    1
1    1
2    1
3    1
4    1
Name: problem_class_encoded, dtype: int64

In [67]:
ALGO_KEYWORDS = [
    "graph", "tree", "dfs", "bfs", "dijkstra",
    "dynamic programming", "dp", "greedy",
    "segment tree", "fenwick", "binary search",
    "two pointers", "union find", "disjoint set",
    "mst", "topological", "bitmask", "backtracking"
]

DS_KEYWORDS = [
    "array", "stack", "queue", "deque",
    "heap", "priority queue", "set",
    "map", "hash", "linked list"
]

ALL_KEYWORDS = ALGO_KEYWORDS + DS_KEYWORDS


In [68]:
data["text"][3]

'the wire ghost žofka is bending a copper wire. she starts with a straight\n    wire placed on the table with the starting point glued to the\n    middle of the table. she then repeatedly picks a point on the\n    wire and bends the part starting at that point (away from the\n    starting point) by $90$\n    degrees (either clockwise or counterclockwise). throughout the\n    process the starting point stays glued to the middle of the\n    table.\nthe most important consideration is that she does not want\n    the wire to touch itself as she bends it (that would summon the\n    wire ghost). she needs your help. she has a list of points\n    together with the direction at each point (clockwise or\n    counterclockwise). she wants to know if bending the wire at the\n    listed points in the given order would cause the wire ghost to\n    appear at any time during the process.\n the first line contains two integers $l$ and $n$ where $l$ is the length of the wire and\n    $n$ is the number o

In [69]:
def extract_numeric_features(text):
    return [
        len(text),                                 
        len(re.findall(r'\d+', text)),             
        len(re.findall(r'10\^\d+', text)),        
        len(re.findall(r'1e\d+', text)),           
        text.count("≤") + text.count("<="),         
        text.count("log"),                         
        text.count("mod"),                          
        len(re.findall(r'O\(.+?\)', text)),       
    ]





In [70]:
numeric_main = np.array(
    [extract_numeric_features(t) for t in data["text"]]
)

numeric_io = np.array(
    [extract_numeric_features(t) for t in data["io_text"]]
)



In [71]:
tfidf_char = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=3,
    max_features=20000
)

X_char = tfidf_char.fit_transform(data["text"])

In [72]:
tfidf_keywords = TfidfVectorizer(
    vocabulary=ALL_KEYWORDS,
    lowercase=True
)

X_keywords = tfidf_keywords.fit_transform(data["text"])


In [73]:
scaler = StandardScaler()
numeric_all = scaler.fit_transform(
    np.hstack([numeric_main, numeric_io])
)


In [74]:
X = hstack([
    X_char,
    X_keywords,
    numeric_all
])

In [75]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_class,
    test_size=0.2,
    random_state=42,
    stratify=y_class
)



In [76]:
from xgboost import XGBClassifier

reg_class = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8
)


reg_class.fit(X_train, y_train)



,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [77]:
y_pred = reg_class.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))



Accuracy: 0.5224787363304981
              precision    recall  f1-score   support

        easy       0.49      0.29      0.36       153
        hard       0.57      0.79      0.66       389
      medium       0.40      0.28      0.33       281

    accuracy                           0.52       823
   macro avg       0.49      0.45      0.45       823
weighted avg       0.50      0.52      0.49       823



In [78]:
y_score = data["problem_score"].values


In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_score,
    test_size=0.2,
    random_state=42
)


In [80]:
from xgboost import XGBRegressor

reg = XGBRegressor(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    objective="reg:squarederror",
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)


In [81]:
reg.fit(X_train, y_train)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.85
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [82]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

y_pred = reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("MAE:", mae)
print("R²:", r2)


RMSE: 1.9920282582417483
MAE: 1.6450063239905264
R²: 0.17333868230094107


In [83]:
import numpy as np
import re
from scipy.sparse import hstack


text = """'house building a number of eccentrics from central new york have decided\n    that they have had enough of modern society, and want to move\n    from there. together they have bought a rectangular piece of\n    land far away, and will now settle there.\nthe land consists of $n \\times\n    m$ squares, and it is possible to build a maximum of one\n    house on a given square. each square has value $a_{x,y}$ that describes how nice it\n    is, on a scale between $0$\n    and $100$.\nthe goal of the eccentrics is to get as far away as possible\n    from everyone else, including each other. the happiness an\n    eccentric experiences from building his house on square\n    $(x,y)$ is thus\n    $a_{x,y}\\cdot d$, where\n    $d$ is the smallest\n    distance to another person.\nout of habit, the eccentrics use manhattan\n    distance to measure this; $d$ is defined as $\\min |x - x_2| + |y - y_2|$ over all\n    other people’s squares $(x_2,\n    y_2)$.\nthe eccentrics now want your help in placing their houses\n    optimally, so that the sum of the happiness they experience is\n    as high as possible. can you help them?\n the input consists of $10$ test cases, which are described\n    below. print $k$ lines with\n    the positions of the houses. each line should contain two\n    numbers: first the row for the house (between $1$ and $n$), then the column (between\n    $1$ and $m$). two houses may not be placed at\n    the same position.'"""


text_clean = text.lower()


io_text = text_clean

def extract_numeric_features(text):
    return [
        len(text),
        len(re.findall(r'\d+', text)),
        len(re.findall(r'10\^\d+', text)),
        len(re.findall(r'1e\d+', text)),
        text.count("≤") + text.count("<="),
        text.count("log"),
        text.count("mod"),
        len(re.findall(r'O\(.+?\)', text))
    ]

numeric_main = np.array([extract_numeric_features(text_clean)])
numeric_io = np.array([extract_numeric_features(io_text)])

numeric_all = scaler.transform(np.hstack([numeric_main, numeric_io]))


X_char_new = tfidf_char.transform([text_clean])
X_keywords_new = tfidf_keywords.transform([text_clean])


X_new = hstack([X_char_new, X_keywords_new, numeric_all])

y_class_pred = reg_class.predict(X_new)
class_name = le.inverse_transform(y_class_pred)[0]


score_value = float(reg.predict(X_new)[0])

print("Predicted class:", class_name)
print("Predicted score:", score_value)





Predicted class: hard
Predicted score: 8.860269546508789


In [84]:
def predict_problem(problem_text, input_text, output_text):
    
    full_text = (problem_text + " " + input_text + " " + output_text).lower()
    io_text = (input_text + " " + output_text).lower()

 
    num_main = np.array([extract_numeric_features(full_text)])
    num_io = np.array([extract_numeric_features(io_text)])

    numeric_all = scaler.transform(
        np.hstack([num_main, num_io])
    )

    
    X_char = tfidf_char.transform([full_text])
    X_keywords = tfidf_keywords.transform([full_text])

   
    X = hstack([X_char, X_keywords, numeric_all])

    
    y_class_pred = reg_class.predict(X_new)
    class_name = le.inverse_transform(y_class_pred)[0]

    score  = float(reg.predict(X_new)[0])


    return class_name, score

In [85]:
predict_problem(
    "Given an array of integers",
    "First line contains n",
    "Print the answer"
)


('hard', 8.860269546508789)

In [86]:
!pip install streamlit



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
!pip install joblib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
import joblib

joblib.dump(reg_class, "reg_class.pkl")
joblib.dump(reg, "reg.pkl")
joblib.dump(tfidf_char, "tfidf_char.pkl")
joblib.dump(tfidf_keywords, "tfidf_keywords.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(le, "label_encoder.pkl")


['label_encoder.pkl']

In [89]:
data["description"][0]

'Unununium (Uuu) was the name of the chemical\n    element with atom number 111, until it changed to\n    Röntgenium (Rg) in 2004. These heavy elements are very\n    unstable and have only been synthesized in a few\n    laboratories.\nYou have just been hired by one of these labs to optimize\n    the algorithms used in simulations. For example, when\n    simulating complicated chemical reactions, it is important to\n    keep track of how many particles there are, and this is done by\n    counting connected components in a graph.\nCurrently, the lab has some Python code (see attachments)\n    that takes an undirected graph and outputs the number of\n    connected components. As you can see, this code is based on\n    everyone’s favourite data structure union-find1.\nAfter looking at the code for a while, you notice that it\n    actually has a bug in it! The code still gives correct answers,\n    but the bug could cause it to run inefficiently. Your task is\n    to construct a graph with

In [90]:
data["input_description"][0]

'The input consists of one line with two integers\n    $N$ and $M$, the number of vertices and edges\n    your graph should have. Apart from the sample, there will be\n    only one test case, with $N =\n    100$ and $M =\n    500$.'

In [91]:
data["output_description"][0]

'The output consists of $M$ lines where the $i$:th contains two integers\n    $u_ i$ and $v_ i$ ($1 \\leq u_ i, v_ i \\leq N$). This\n    indicates that the vertices $u_\n    i$ and $v_ i$ are\n    connected with an edge in your graph.'